In [1]:
import pandas as pd
pd.set_option("display.max_row", None)
%load_ext autoreload
%autoreload 2

In [2]:
from absbox import API,mkDeal
localAPI = API("http://localhost:8081",check=False)

✅Connected, local lib:0.25.0, server:0.25.0

In [82]:
deal_data = {
    "name":"Multiple Pools"
    ,"dates":{"cutoff":"2021-06-01"
              ,"closing":"2021-07-15"
              ,"firstPay":"2021-08-26"
              ,"payFreq":["DayOfMonth",20]
              ,"poolFreq":"MonthEnd"
              ,"stated":"2030-01-01"}
    ,"pool":{"PoolA":{'assets':[["Mortgage"
                        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
                          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                          ,{"currentBalance":2200
                          ,"currentRate":0.08
                          ,"remainTerm":30
                          ,"status":"current"}]]},
             "PoolB":{'assets':[["Mortgage"
                        ,{"originBalance":1100,"originRate":["fix",0.045],"originTerm":30
                          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
                          ,{"currentBalance":1100
                          ,"currentRate":0.08
                          ,"remainTerm":30
                          ,"status":"current"}]]}
            }
    ,"accounts":{"acc01":{"balance":0}
                ,"acc02":{"balance":0}}
    ,"bonds":{"A1":{"balance":1000
                 ,"rate":0.07
                 ,"originBalance":1000
                 ,"originRate":0.07
                 ,"startDate":"2020-01-03"
                 ,"rateType":{"Fixed":0.08}
                 ,"bondType":{"Sequential":None}}
             ,"B":{"balance":1000
                     ,"rate":0.0
                     ,"originBalance":1000
                     ,"originRate":0.07
                     ,"startDate":"2020-01-03"
                     ,"rateType":{"Fixed":0.00}
                     ,"bondType":{"Equity":None}}}
    ,"fees":{"trusteeFee":{"type":{"fixFee":30}}
            ,"serviceFee":{"type":{"annualPctFee":[("poolBalance","PoolB"),0.02]}}
            ,"serviceFee2":{"type":{"byTable":["MonthEnd"
                                               ,("const",1)
                                               ,[(0,5),(2,10),(10,15)]]
                                   }}}
    ,"collect":[[["PoolA"],"CollectedCash","acc01"]
               ,[["PoolB"],"CollectedCash","acc02"]]
    ,"waterfall":{"Amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["calcAndPayFee","acc01",['serviceFee']]
         ,["payInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,"status":("PreClosing","Amortizing")
}
from absbox import mkDeal
deal = mkDeal(deal_data)  ## now a generic class created

In [91]:
deal.json['contents']['pool']

{'tag': 'MultiPool',
 'contents': {'PoolName:PoolA': {'assets': [{'tag': 'Mortgage',
     'contents': [{'originBalance': 2200,
       'originRate': {'tag': 'Fix', 'contents': ['DC_ACT_365F', 0.045]},
       'originTerm': 30,
       'period': 'Monthly',
       'startDate': '2021-02-01',
       'prinType': {'tag': 'Level'},
       'prepaymentPenalty': None,
       'tag': 'MortgageOriginalInfo'},
      2200,
      0.08,
      30,
      None,
      {'tag': 'Current'}]}],
   'asOfDate': '2021-06-01',
   'issuanceStat': None,
   'futureCf': None,
   'extendPeriods': {'tag': 'MonthEnd'}},
  'PoolName:PoolB': {'assets': [{'tag': 'Mortgage',
     'contents': [{'originBalance': 1100,
       'originRate': {'tag': 'Fix', 'contents': ['DC_ACT_365F', 0.045]},
       'originTerm': 30,
       'period': 'Monthly',
       'startDate': '2021-02-01',
       'prinType': {'tag': 'Level'},
       'prepaymentPenalty': None,
       'tag': 'MortgageOriginalInfo'},
      1100,
      0.08,
      30,
      None,
 

In [97]:
r = localAPI.run(deal
                 ,poolAssump = ("ByName"
                                ,{"PoolA":
                                  (("Mortgage",{"CDR":0.02} ,None, None, None)
                                   ,None
                                   ,None)})
                 ,runAssump = [("interest",("LIBOR6M",0.04))]
                 ,read=True)

Warning Message from server:
Fee serviceFee2 is not paid off
Account acc01 has cash to be distributed
Account acc02 has cash to be distributed

In [66]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-08-26,881.80,12.08,118.20,0.07,130.28,"[<PayInt:A1>, <PayPrin:A1>]"
2021-09-20,808.05,6.34,73.75,0.07,80.09,"[<PayInt:A1>, <PayPrin:A1>]"
2021-10-20,735.31,6.97,72.74,0.07,79.71,"[<PayInt:A1>, <PayPrin:A1>]"
2021-11-20,662.30,6.55,73.01,0.07,79.56,"[<PayInt:A1>, <PayPrin:A1>]"
2021-12-20,588.53,5.71,73.77,0.07,79.48,"[<PayInt:A1>, <PayPrin:A1>]"
2022-01-20,514.44,5.24,74.09,0.07,79.33,"[<PayInt:A1>, <PayPrin:A1>]"
2022-02-20,439.81,4.58,74.63,0.07,79.21,"[<PayInt:A1>, <PayPrin:A1>]"
2022-03-20,364.13,3.54,75.68,0.07,79.22,"[<PayInt:A1>, <PayPrin:A1>]"
2022-04-20,288.37,3.24,75.76,0.07,79.00,"[<PayInt:A1>, <PayPrin:A1>]"


In [62]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-08-26,877.77,8.05,122.23,0.07,130.28,"[<PayInt:A1>, <PayPrin:A1>]"
2021-09-20,801.88,4.20,75.89,0.07,80.09,"[<PayInt:A1>, <PayPrin:A1>]"
2021-10-20,726.78,4.61,75.10,0.07,79.71,"[<PayInt:A1>, <PayPrin:A1>]"
2021-11-20,651.54,4.32,75.24,0.07,79.56,"[<PayInt:A1>, <PayPrin:A1>]"
2021-12-20,575.80,3.74,75.74,0.07,79.48,"[<PayInt:A1>, <PayPrin:A1>]"
2022-01-20,499.89,3.42,75.91,0.07,79.33,"[<PayInt:A1>, <PayPrin:A1>]"
2022-02-20,423.65,2.97,76.24,0.07,79.21,"[<PayInt:A1>, <PayPrin:A1>]"
2022-03-20,346.70,2.27,76.95,0.07,79.22,"[<PayInt:A1>, <PayPrin:A1>]"
2022-04-20,269.76,2.06,76.94,0.07,79.00,"[<PayInt:A1>, <PayPrin:A1>]"
